# Description

Many Python packages contain numerous submodules, or even deeper nesting.  Within those various levels of nesting, various kinds of objects occur.  Of particular interest to us are functions, class, and scalars.

You job is to write a function call `describe()` that will produce a hierarchical report of an overall package.  For example:

```python
>>> import textwrap
>>> describe(textwrap, module_names_only=True)
{'modules': {'re': '...'},
 'functions': {'dedent': '<Signature (text)>',
  'fill': '<Signature (text, width=70, **kwargs)>',
  'indent': '<Signature (text, prefix, predicate=None)>',
  'shorten': '<Signature (text, width, **kwargs)>',
  'wrap': '<Signature (text, width=70, **kwargs)>'},
 'classes': {'TextWrapper': ['TextWrapper', 'object']},
 'scalars': {'__cached__': '/home/dmertz/minicon...',
  '__doc__': 'Text wrapping and fi...',
  '__file__': '/home/dmertz/minicon...',
  '__name__': 'textwrap',
  '__package__': '',
  '_whitespace': '\t\n\x0b\x0c\r '}}
```

# Setup

In [1]:
import inspect

def describe(mod):
    return dict()

# Solution

In [5]:
def isscalar(obj):
    return isinstance(obj, (int, complex, float, str, bytes))

    
def describe(obj, module_names_only=False, seen=None):
    seen = set() if seen is None else seen
    summary = dict(modules=dict(), 
                   functions=dict(), 
                   classes=dict(), 
                   scalars=dict())
    
    # Circular import graphs are a problem
    for submod in inspect.getmembers(obj, inspect.ismodule):
        if module_names_only:
            summary['modules'][submod[0]] = "..."
        elif not repr(submod) in seen:
            summary['modules'][submod[0]] = describe(submod[1], seen)
            seen.add(repr(submod))
            
    for func in inspect.getmembers(obj, inspect.isfunction):
        try:
            summary['functions'][func[0]] = repr(inspect.signature(func[1]))
        except:
            summary['functions'][func[0]] = None
            
    for klass in inspect.getmembers(obj, inspect.isclass):
        summary['classes'][klass[0]] = [k.__name__ for k in inspect.getmro(klass[1])]
        
    for scalar in inspect.getmembers(obj, isscalar):
        val = scalar[1]
        if isinstance(val, str):
            if len(val) > 20:
                val = val[:20] + "..."
        summary['scalars'][scalar[0]] = val
        
    return summary

# Test Cases

In [4]:
def test_textwrap():
    import textwrap
    desc = describe(textwrap, module_names_only=True)
    del desc['scalars']['__cached__']
    del desc['scalars']['__file__']
    correct = {
        'modules': {'re': '...'},
        'functions': {'dedent': '<Signature (text)>',
        'fill': '<Signature (text, width=70, **kwargs)>',
        'indent': '<Signature (text, prefix, predicate=None)>',
        'shorten': '<Signature (text, width, **kwargs)>',
        'wrap': '<Signature (text, width=70, **kwargs)>'},
        'classes': {'TextWrapper': ['TextWrapper', 'object']},
        'scalars': {'__doc__': 'Text wrapping and fi...',
        '__name__': 'textwrap',
        '__package__': '',
        '_whitespace': '\t\n\x0b\x0c\r '}}
    assert desc == correct
    
test_textwrap()

In [10]:
def test_asyncio():
    import asyncio
    submods = set(describe(asyncio)['modules'])
    correct = {'base_events', 'base_futures', 'base_subprocess', 'base_tasks',
               'constants', 'coroutines', 'events', 'exceptions', 
               'format_helpers', 'futures', 'locks', 'log', 'protocols',
               'queues', 'runners', 'selector_events', 'sslproto', 'staggered',
               'streams', 'subprocess', 'sys', 'tasks', 'transports', 'trsock',
               'unix_events'} 
    assert submods == correct
    
test_asyncio()

In [17]:
def test_re():
    import re
    names = set(describe(re)['scalars'])
    correct = {'A', 'ASCII', 'DEBUG', 'DOTALL', 'I', 'IGNORECASE', 'L',
               'LOCALE', 'M', 'MULTILINE', 'S', 'T', 'TEMPLATE', 'U',
               'UNICODE', 'VERBOSE', 'X'}
    assert correct <= names

test_re()